### Code for new data

In [ ]:
#Loading necessary libraries
import numpy as np
import pandas as pd
import xarray as xr
from typing import List, Dict

#Defining the MRRStormDetector class

class MRRStormDetector:
    """
    Storm detection and melting–layer retrieval for MRR data,
    including quality control filtering and 10-minute segmented
    moving average calculation for melting layer within each event.
    """

    def __init__(self,
                 gap_threshold_h: float = 3.0,
                 detection_h_m: float = 450.0,
                 gate_spacing_m: float = 100.0,
                 melt_max_h_m: float = 1500.0,
                 min_duration_h: float = 0.75,  # 45 minutes minimum
                 min_gates_per_minute: int = 15,
                 segment_duration_min: float = 10.0):  # 10-minute segments
        self.gap_threshold_h = gap_threshold_h
        self.detection_h_m = detection_h_m
        self.gate_spacing_m = gate_spacing_m
        self.melt_max_h_m = melt_max_h_m
        self.min_duration_h = min_duration_h
        self.min_gates_per_minute = min_gates_per_minute
        self.segment_duration_min = segment_duration_min

    def load_dataset(self, filepath: str) -> xr.Dataset:
        return xr.open_dataset(filepath)

    def find_detection_gate(self, ds: xr.Dataset) -> int:
        heights = (np.arange(ds.dims["range"]) + 1) * self.gate_spacing_m
        return int(np.argmin(np.abs(heights - self.detection_h_m)))

    def make_precip_mask(self, refl: xr.DataArray, gate_idx: int) -> np.ndarray:
        gate_refl = refl.isel(range=gate_idx)
        return (~np.isnan(gate_refl)).values

    def detect_melting_layer(self,
                             refl: xr.DataArray,
                             vel: xr.DataArray,
                             precip_mask: np.ndarray,
                             gate_mask_offset: int = 2) -> np.ndarray:
        T, G = refl.shape
        melt_heights = np.full(T, np.nan)
        max_gate = int(self.melt_max_h_m // self.gate_spacing_m)
        gate_slice = slice(gate_mask_offset, max_gate)

        dbz = refl.values
        w = vel.values

        for t in np.where(precip_mask)[0]:
            z_profile = dbz[t, gate_slice]
            w_profile = w[t, gate_slice]
            if z_profile.size == 0 or w_profile.size == 0:
                continue
            if np.all(np.isnan(z_profile)) or np.all(np.isnan(w_profile)):
                continue

            peak_gate_rel = int(np.nanargmax(z_profile))
            w_grad = np.diff(w_profile)
            if w_grad.size == 0 or np.all(np.isnan(w_grad)):
                continue
            accel_gate_rel = int(np.nanargmax(w_grad))

            if abs(peak_gate_rel - accel_gate_rel) <= 1:
                best_gate_rel = int(round((peak_gate_rel + accel_gate_rel) / 2))
            else:
                best_gate_rel = peak_gate_rel

            best_gate_abs = gate_mask_offset + best_gate_rel
            melt_heights[t] = (best_gate_abs + 1) * self.gate_spacing_m

        return melt_heights

    def calculate_event_segments(self, 
                                start_time: pd.Timestamp, 
                                end_time: pd.Timestamp, 
                                segment_duration_min: float) -> List[Dict]:
        segments = []
        segment_delta = pd.Timedelta(minutes=segment_duration_min)
        current_start = start_time
        
        while current_start < end_time:
            current_end = min(current_start + segment_delta, end_time)
            segments.append({
                'segment_start': current_start,
                'segment_end': current_end,
                'segment_duration_min': (current_end - current_start).total_seconds() / 60.0
            })
            current_start = current_end
            
        return segments

    def calculate_segment_melting_layer_averages(self,
                                               melt_heights: np.ndarray,
                                               times: np.ndarray,
                                               start_idx: int,
                                               end_idx: int) -> List[float]:

        # Extract event data
        event_melt_heights = melt_heights[start_idx:end_idx+1]
        event_times = times[start_idx:end_idx+1]
        times_dt = pd.to_datetime(event_times)
        
        # Get event start and end times
        start_time = times_dt[0]
        end_time = times_dt[-1]
        
        # Calculate segments
        segments = self.calculate_event_segments(start_time, end_time, self.segment_duration_min)
        
        segment_averages = []
        for segment in segments:
            # Find data points within this segment
            mask = (times_dt >= segment['segment_start']) & (times_dt <= segment['segment_end'])
            segment_melt_data = event_melt_heights[mask]
            
            # Calculate average, ignoring NaN values
            valid_data = segment_melt_data[~np.isnan(segment_melt_data)]
            if len(valid_data) > 0:
                segment_avg = float(np.mean(valid_data))
            else:
                segment_avg = np.nan
                
            segment_averages.append(segment_avg)
            
        return segment_averages

    def identify_events(self, mask: np.ndarray, times: np.ndarray) -> List[Dict]:
        times_dt = pd.to_datetime(times)
        idxs = np.where(mask)[0]
        if idxs.size == 0:
            return []

        events, start, end = [], idxs[0], idxs[0]
        for i in idxs[1:]:
            gap_h = (times_dt[i] - times_dt[end]).total_seconds() / 3600.0
            if gap_h <= self.gap_threshold_h:
                end = i
            else:
                events.append((start, end))
                start = end = i
        events.append((start, end))

        return [
            dict(row_start=s,
                 row_end=e,
                 start_time=times_dt[s],
                 end_time=times_dt[e],
                 duration_h=(times_dt[e] - times_dt[s]).total_seconds() / 3600.0)
            for s, e in events
        ]

    def apply_quality_filters(self, 
                             events: List[Dict], 
                             refl_data: xr.DataArray) -> List[Dict]:
        """
        Applying quality control filters to remove events with insufficient 
        duration or data availability.
        """
        if not events:
            return []
            
        initial_count = len(events)
        duration_filtered = []
        
        # Filter 1: Duration check
        for event in events:
            if event["duration_h"] >= self.min_duration_h:
                duration_filtered.append(event)
        
        duration_removed = initial_count - len(duration_filtered)
        
        # Filter 2: Data availability check
        data_filtered = []
        for event in duration_filtered:
            s, e = event["row_start"], event["row_end"]
            event_data = refl_data.isel(time=slice(s, e+1))
            
            has_sufficient_data = False
            for t in range(event_data.shape[0]):
                valid_gates = np.sum(~np.isnan(event_data.values[t, :]))
                if valid_gates >= self.min_gates_per_minute:
                    has_sufficient_data = True
                    break
                    
            if has_sufficient_data:
                data_filtered.append(event)
        
        data_removed = len(duration_filtered) - len(data_filtered)
        total_removed = initial_count - len(data_filtered)
        
        print(f"Quality control filtering results:")
        print(f"  Initial events: {initial_count}")
        print(f"  Removed by duration filter (<{self.min_duration_h:.2f}h): {duration_removed}")
        print(f"  Removed by data availability filter (<{self.min_gates_per_minute} gates): {data_removed}")
        print(f"  Final events after filtering: {len(data_filtered)}")
        print(f"  Total removal rate: {total_removed/initial_count*100:.1f}%")
            
        return data_filtered

    def classify_events(self,
                        ds: xr.Dataset,
                        events: List[Dict],
                        refl_var: str,
                        melt_heights: np.ndarray) -> List[Dict]:
        enhanced = []
        times = ds.time.values
        
        for i, ev in enumerate(events, 1):
            s, e = ev["row_start"], ev["row_end"]
            seg = ds[refl_var].isel(time=slice(s, e+1))
            mean_ = float(seg.mean().item())
            max_ = float(seg.max().item())
            
            med_m = float(np.nanmedian(melt_heights[s:e+1]))
            
            # Calculating 10-minute segment averages for this event
            segment_averages = self.calculate_segment_melting_layer_averages(
                melt_heights, times, s, e
            )
            
            # Calculating number of 10-minute segments
            num_segments = len(segment_averages)

            # Keeping existing classification logic unchanged
            if max_ > 30 and ev["duration_h"] < 4:
                stype = "convective"
            elif ev["duration_h"] > 8 and max_ < 25:
                stype = "stratiform"
            else:
                stype = "mixed"

            enhanced.append({**ev,
                             "storm_id": i,
                             "mean_dbz": round(mean_, 2),
                             "max_dbz": round(max_, 2),
                             "storm_type": stype,
                             "median_melting_layer_m": round(med_m, 1),
                             "num_10min_segments": num_segments,
                             "segment_ml_averages": [round(avg, 1) if not np.isnan(avg) else None 
                                                   for avg in segment_averages]})
        return enhanced

    def run(self,
            filepath: str,
            refl_var: str = "Ze",
            vel_var: str = "W",
            time_coord: str = "time") -> List[Dict]:
        print(f"Loading dataset: {filepath}")
        ds = self.load_dataset(filepath)
        for var in (refl_var, vel_var):
            if var not in ds:
                raise KeyError(f"Variable '{var}' not found in file.")

        print("Detecting precipitation and melting layers...")
        gate_idx = self.find_detection_gate(ds)
        precip_mask = self.make_precip_mask(ds[refl_var], gate_idx)
        melt_ht = self.detect_melting_layer(ds[refl_var],
                                           ds[vel_var],
                                           precip_mask,
                                           gate_mask_offset=2)
        
        print("Identifying precipitation events...")
        events = self.identify_events(precip_mask, ds[time_coord].values)
        
        print(f"Applying quality control filters...")
        filtered_events = self.apply_quality_filters(events, ds[refl_var])
        
        print("Classifying storm events with 10-minute segment analysis...")
        storms = self.classify_events(ds, filtered_events, refl_var, melt_ht)
        return storms


if __name__ == "__main__":
    detector = MRRStormDetector(
        gap_threshold_h=3.0,
        detection_h_m=450.0,
        gate_spacing_m=100.0,
        melt_max_h_m=1500.0,
        min_duration_h=0.75,  # 45 minutes
        min_gates_per_minute=15,
        segment_duration_min=10.0  # 10-minute segments
    )

    storms = detector.run(
        filepath="C:\\Working Folder\\Complete_Data\\merged_output_new.nc",
        refl_var="Ze",
        vel_var="W",
        time_coord="time"
    )

    print(f"\nFinal results - Detected {len(storms)} quality-controlled storms:")
    for s in storms:
        print(f"  • Storm {s['storm_id']:2d}: {s['start_time']} → {s['end_time']}, "
              f"d={s['duration_h']:.1f} h, type={s['storm_type']}, "
              f"median ML={s['median_melting_layer_m']:.0f} m")
        print(f"    10-min segments ({s['num_10min_segments']}): {s['segment_ml_averages']}")

    # Creating detailed output with expanded segment data
    output_data = []
    for storm in storms:
        base_data = {k: v for k, v in storm.items() if k != 'segment_ml_averages'}
        
        # Adding each segment as a separate column
        for i, avg in enumerate(storm['segment_ml_averages'], 1):
            base_data[f'segment_{i}_ml_avg'] = avg
            
        output_data.append(base_data)

    output_file = r"C:\Working Folder\Summer_Work\MRR\new_data_storm_summary_10min_segments.csv"
    pd.DataFrame(output_data).to_csv(output_file, index=False)
    print(f"Saved 10-minute segment analysis → {output_file}")


Loading dataset: C:\Working Folder\Complete_Data\merged_output_new.nc
Detecting precipitation and melting layers...


C:\Users\icecap\AppData\Local\Temp\ipykernel_18228\1857926043.py:39: FutureWarning: The return type of `Dataset.dims` will be changed to return a set of dimension names in future, in order to be more consistent with `DataArray.dims`. To access a mapping from dimension names to lengths, please use `Dataset.sizes`.
  heights = (np.arange(ds.dims["range"]) + 1) * self.gate_spacing_m


Identifying precipitation events...
Applying quality control filters...
Quality control filtering results:
  Initial events: 207
  Removed by duration filter (<0.75h): 37
  Removed by data availability filter (<15 gates): 14
  Final events after filtering: 156
  Total removal rate: 24.6%
Classifying storm events with 10-minute segment analysis...

Final results - Detected 156 quality-controlled storms:
  • Storm  1: 2024-12-28 12:26:00 → 2024-12-29 03:47:00, d=15.3 h, type=stratiform, median ML=900 m
    10-min segments (93): [1283.3, 1325.0, None, None, None, None, None, None, None, 1433.3, None, 1500.0, 1377.8, 1311.1, 880.0, 845.5, 1054.5, 1350.0, 1450.0, 981.8, 745.5, 590.9, 772.7, 981.8, 627.3, 636.4, 790.9, 1136.4, 1436.4, 1154.5, 590.9, 1245.5, 1345.5, 1172.7, 445.5, 763.6, 1272.7, 1000.0, 972.7, 1100.0, 836.4, 927.3, 954.5, 990.9, 1045.5, 727.3, 590.9, 481.8, 509.1, 354.5, 300.0, 300.0, 1190.9, 1163.6, 872.7, 1154.5, 1200.0, 1045.5, 336.4, 318.2, 990.9, 727.3, 436.4, 1072.7, 12